In order to run this notebook, you will need to download the raw data from the [DNCP website](https://www.contrataciones.gov.py/datos/adjudicaciones).
- We named each folder as "DNCP_Adjudicaciones_*year*" after downloading, depending on the the year of the data it contained. Ex.: folder with data of 2020, was named "DNCP_Adjudicaciones_2020".

In [1]:
import pandas as pd
import os
import re

## Column Extraction with its examples

This section is just to extract the columns of the files in a document that also contains an example of the columns

In [2]:
ubi = '../data/raw_data/DNCP_Adjudicaciones_2020' #change for real location of the raw data folder

In [3]:
files = os.listdir(ubi)

In [7]:
%cd ../data/raw_data/DNCP_Adjudicaciones_2020

In [9]:
pattern = '.+\.csv'
re_pattern = re.compile(pattern)
c_files = [file_name for file_name in files if re_pattern.search(file_name)]

In [10]:
doc = []
col = []
example = []


for file_name in c_files:
    file = pd.read_csv('./'+file_name)
    if file_name not in ['awa_items.csv', 'records.csv', 'awards.csv']:
        file.dropna(inplace=True)
        file.reset_index(inplace=True)
    col += file.columns.to_list()
    doc += [file_name]*len(file.columns)
    example += file.iloc[0,:].to_list()

df = pd.DataFrame({'Document':doc, 'Column':col, 'Example':example})
df.head()

/var/folders/_6/l3fpk3z10lb5g2csy8ykd4lw0000gn/T/ipykernel_94852/79128179.py:7: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  file = pd.read_csv('./'+file_name)
/var/folders/_6/l3fpk3z10lb5g2csy8ykd4lw0000gn/T/ipykernel_94852/79128179.py:7: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  file = pd.read_csv('./'+file_name)
/var/folders/_6/l3fpk3z10lb5g2csy8ykd4lw0000gn/T/ipykernel_94852/79128179.py:7: DtypeWarning: Columns (60,61,62,63) have mixed types. Specify dtype option on import or set low_memory=False.
  file = pd.read_csv('./'+file_name)


,Document,Column,Example
0,awa_ite_attributes.csv,index,0
1,awa_ite_attributes.csv,compiledRelease/id,373381-adquisicion-velaglucerasa-alfa-algasida...
2,awa_ite_attributes.csv,compiledRelease/awards/0/id,373381-laboratorio-productos-eticos-c-e-i-s-a-2
3,awa_ite_attributes.csv,compiledRelease/awards/0/items/0/id,3zVmuGB38TXDlc8qb93fww==
4,awa_ite_attributes.csv,compiledRelease/awards/0/items/0/attributes/0/id,1


In [13]:
df.to_excel('DNCP_columns.xlsx', index=False)

# Data Wrangling

In [16]:
ubi = '../' #change with the directory of the folders
ubi_suppliers = 'awa_suppliers.csv'
ubi_records = 'records.csv'
ubi_awards = 'awards.csv'

important_columns = '../DNCP_columnas.xlsx'

folder_name = 'DNCP_Adjudicaciones_'

start_year = 2013
end_year = 2023

In [17]:
col_data = pd.read_excel(ubi+important_columns)
col_data = col_data[col_data['Compile']=='y']

dic_col = {}

for row in range(col_data.shape[0]):
    doc = col_data.loc[row, 'Document']
    col = col_data.loc[row, 'Column']
    if doc in dic_col:
        dic_col[doc].append(col)
    else:
        dic_col[doc] = [col]

### Institution - Supplier - Amount

#### Filtering - Supplier

In [18]:
re_pattern = re.compile('PY-RUC.+')

In [19]:
### Paraguayan organization

In [20]:
supplier_origin = {}
total_suppliers = 0
for year in range(start_year,end_year+1):
    suppliers = pd.read_csv(ubi+folder_name+str(year)+'/'+ubi_suppliers)
    suppliers = suppliers[dic_col[ubi_suppliers]]
    suppliers = suppliers.drop_duplicates(['compiledRelease/id','compiledRelease/awards/0/suppliers/0/id'])

    sup_orig = suppliers['compiledRelease/awards/0/suppliers/0/id'].map(lambda row: bool(re_pattern.search(row)))
    sup_pyo = sup_orig.sum()
    sup_for = sup_orig.count() - sup_pyo
    
    supplier_origin[year] = [sup_pyo, sup_for]
    total_suppliers += sup_pyo
    
print(supplier_origin)
print(f'Total Paraguayan Suppliers: {total_suppliers}')

{2013: [29788, 40], 2014: [13376, 42], 2015: [11241, 37], 2016: [11357, 25], 2017: [12097, 24], 2018: [11508, 32], 2019: [30045, 124], 2020: [7395, 20], 2021: [7815, 29], 2022: [25554, 36], 2023: [16468, 32]}
Total Paraguayan Suppliers: 176644


In [136]:
first_data = True
for year in range(start_year, end_year+1):
    suppliers = pd.read_csv(ubi+folder_name+str(year)+'/'+ubi_suppliers)
    suppliers = suppliers[dic_col[ubi_suppliers]]
    suppliers['Paraguayan'] = suppliers['compiledRelease/awards/0/suppliers/0/id'] \
                              .map(lambda row: bool(re_pattern.search(row)))
    suppliers['Year'] = year
    suppliers = suppliers[suppliers['Paraguayan']==True]

    
    if first_data:
        all_suppliers = suppliers.copy()
        first_data = False
    
    else:
        all_suppliers = pd.concat([all_suppliers, suppliers])

all_suppliers = all_suppliers.drop_duplicates(['compiledRelease/id','compiledRelease/awards/0/suppliers/0/id'])
all_suppliers.shape

(176644, 6)

In [141]:
all_suppliers.to_csv('../../clean_supplers.csv')

#### Filtering - Awards Records

In [149]:
first_data = True
for year in range(start_year, end_year+1):
    records = pd.read_csv(ubi+folder_name+str(year)+'/'+ubi_records)
    records = records[['compiledRelease/buyer/id','compiledRelease/buyer/name', 
                   'compiledRelease/tender/mainProcurementCategoryDetails',
                  'compiledRelease/id']]
    records['Year'] = year
    
    if first_data:
        all_records = records.copy()
        first_data = False
    
    else:
        all_records = pd.concat([all_records, records])
        
all_records = all_records.drop_duplicates(['compiledRelease/id', 'compiledRelease/buyer/id'])

#elimina los que no tienen categoría en el servicio
all_records = all_records[pd.notna(all_records['compiledRelease/tender/mainProcurementCategoryDetails'])]

#elimina los que no tienen institución compradora
all_records = all_records[pd.notna(all_records['compiledRelease/buyer/id'])]

all_records.shape

/Users/matiasromeromoriya/miniconda3/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3460: DtypeWarning: Columns (60,61,62,63) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/matiasromeromoriya/miniconda3/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3460: DtypeWarning: Columns (64,65) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/matiasromeromoriya/miniconda3/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3460: DtypeWarning: Columns (62,63,64,65) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/matiasromeromoriya/miniconda3/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3460: DtypeWarning: Columns (62,63) have mixed types.Specify dtype option on import or set low_memory=False.


(197504, 5)

In [150]:
all_records.to_csv('../../clean_records.csv')

In [173]:
all_records['compiledRelease/buyer/name'].value_counts()

Ministerio de Salud Pública y Bienestar Social (MSPBS)    10270
Universidad Nacional de Asunción (UNA)                     9604
Presidencia de la República                                9047
Corte Suprema de Justicia (CSJ)                            6509
Ministerio de Defensa Nacional (MDN)                       6173
                                                          ...  
Municipalidad de Campo Aceval                                17
Municipalidad de San Jose Del Rosario                        17
Instituto Nacional del Audiovisual Paraguayo (INAP)          15
Dirección Nacional de Vigilancia Sanitaria  (DINAVISA)       15
Municipalidad de Itacua                                       6
Name: compiledRelease/buyer/name, Length: 375, dtype: int64

In [174]:
all_records['compiledRelease/buyer/id'].value_counts()

DNCP-SICP-CODE-306    10270
DNCP-SICP-CODE-226     9604
DNCP-SICP-CODE-301     9047
DNCP-SICP-CODE-21      6509
DNCP-SICP-CODE-304     6173
                      ...  
DNCP-SICP-CODE-383       17
DNCP-SICP-CODE-384       17
DNCP-SICP-CODE-386       15
DNCP-SICP-CODE-390       15
DNCP-SICP-CODE-387        6
Name: compiledRelease/buyer/id, Length: 375, dtype: int64

#### Filtering - Awards details

In [21]:
awards = pd.read_csv(ubi+folder_name+str(2020)+'/'+ubi_awards)

In [22]:
awards = awards[dic_col[ubi_awards]]

In [23]:
awards = awards[(awards['compiledRelease/awards/0/statusDetails']=='Adjudicado') & 
                (awards['compiledRelease/awards/0/value/currency']=='PYG')]

In [24]:
awards.drop_duplicates(['compiledRelease/awards/0/id','compiledRelease/id'])

,compiledRelease/awards/0/statusDetails,compiledRelease/awards/0/value/amount,compiledRelease/awards/0/value/currency,compiledRelease/awards/0/id,compiledRelease/id
0,Adjudicado,3720131184,PYG,373381-laboratorio-productos-eticos-c-e-i-s-a-2,373381-adquisicion-velaglucerasa-alfa-algasida...
1,Adjudicado,506063415,PYG,371497-luis-dario-benitez-jara-1,371497-construccion-pavimento-tipo-empedrado-a...
2,Adjudicado,4607750,PYG,371508-fulgencio-ramon-barcelo-cabrera-1,371508-construccion-camara-septica-drenaje-agu...
3,Adjudicado,10000000,PYG,371679-mara-lia-gonzalez-cabrera-1,371679-elaboracion-proyecto-inversion-ciudad-a...
4,Adjudicado,3126415914,PYG,371719-miguel-vicente-zarate-irala-2,371719-restauracion-espacios-publicos-1-157918...
...,...,...,...,...,...
7500,Adjudicado,300562300,PYG,376576-comtel-sociedad-anonima-5,376576-lp1576-20-adquisicion-equipos-control-e...
7501,Adjudicado,351698100,PYG,376576-data-lab-sa-3,376576-lp1576-20-adquisicion-equipos-control-e...
7502,Adjudicado,200000000,PYG,376597-alejandro-recalde-mora-2,376597-cambio-alfombras-persianas-mamparas-1-1...
7503,Adjudicado,200000000,PYG,376681-nancy-larroza-insaurralde-1,376681-adquisicion-ambulancia-1-1607338081


In [170]:
first_data = True
for year in range(start_year, end_year+1):
    awards = pd.read_csv(ubi+folder_name+str(year)+'/'+ubi_awards)
    awards = awards[dic_col[ubi_awards]]
    awards['Year'] = year
    awards = awards[(awards['compiledRelease/awards/0/statusDetails']=='Adjudicado') & 
                (awards['compiledRelease/awards/0/value/currency']=='PYG')]
    
    if first_data:
        all_awards = awards.copy()
        first_data = False
    
    else:
        all_awards = pd.concat([all_awards, awards])
        
all_awards.shape

(177822, 6)

In [178]:
all_awards

,compiledRelease/awards/0/statusDetails,compiledRelease/awards/0/value/amount,compiledRelease/awards/0/value/currency,compiledRelease/awards/0/id,compiledRelease/id,Year
0,Adjudicado,416301530,PYG,245280-b-braun-medical-paraguay-s-a-37,245280-lpn-06-12-adquisicion-insumos-regiones-...,2013
1,Adjudicado,123052000,PYG,245280-cateteres-afines-s-a-35,245280-lpn-06-12-adquisicion-insumos-regiones-...,2013
2,Adjudicado,944559860,PYG,245280-chaco-internacional-sa-26,245280-lpn-06-12-adquisicion-insumos-regiones-...,2013
3,Adjudicado,218202620,PYG,245280-codex-s-r-l-32,245280-lpn-06-12-adquisicion-insumos-regiones-...,2013
4,Adjudicado,10965400,PYG,245280-d-d-distribuidora-sa-33,245280-lpn-06-12-adquisicion-insumos-regiones-...,2013
...,...,...,...,...,...,...
37739,Adjudicado,68038000,PYG,429626-francisco-sabino-rotela-lopez-1,429626-adquisicion-mobiliarios-instituciones-e...,2023
37740,Adjudicado,88674120,PYG,430097-alsa-s-a-1,430097-construccion-capilla-cruz-mayor-1-16958...,2023
37741,Adjudicado,1564062569,PYG,430974-eduardo-agustin-1,430974-trabajos-mejoras-usf-distritos-departam...,2023
37742,Adjudicado,89757362,PYG,432465-rosana-mayumi-kasuga-miranda-1,432465-terminacion-sanitario-paseo-teros-1-169...,2023


In [179]:
all_awards.to_csv('../../clean_awards.csv')